In [211]:
import os
import json
import pandas as pd
import requests
import time
import datetime
from datetime import date
import calendar
import csv

In [212]:
# date

after_start_date = '2016-04'
# before_end_date = '2019-04'
before_end_date = '2019-04'

def get_start_end_day(year, month):
    _, monthRange = calendar.monthrange(year, month)
    first_day = datetime.date(year=year, month=month, day=1)
    last_day = datetime.date(year=year, month=month, day=monthRange)
    return first_day, last_day

def get_each_month_in_a_list(after_start_date, before_end_date):
    month_pair_list = []
    start_year, start_month = after_start_date.split('-')
    end_year, end_month = before_end_date.split('-')
    start_year, start_month, end_year, end_month = int(start_year), int(start_month), int(end_year), int(end_month)
    
    temp_year, temp_month = start_year, start_month
    while(temp_year<=end_year):
        if(temp_year<end_year):
            while(temp_month<=12):
                first_day_of_month, last_day_of_month = get_start_end_day(temp_year, temp_month)
                month_pair_list.append((first_day_of_month, last_day_of_month))
                temp_month += 1
        else:
            while(temp_month<=end_month):
                first_day_of_month, last_day_of_month = get_start_end_day(temp_year, temp_month)
                month_pair_list.append((first_day_of_month, last_day_of_month))
                temp_month += 1
        temp_month = temp_month%12
        temp_year += 1
    return month_pair_list

def get_start_end_seconds(after_start_date, before_end_date):
    after_start_seconds = (after_start_date -  date.fromisoformat('1970-01-01')).total_seconds()
    before_end_seconds = (before_end_date -  date.fromisoformat('1970-01-01')).total_seconds()
    return after_start_seconds, before_end_seconds

def get_each_month_second_in_a_list(month_pair_list):
    month_second_pair_list = []
    for after_start_date, before_end_date in month_pair_list:
        after_start_seconds, before_end_seconds = get_start_end_seconds(after_start_date, before_end_date)
        month_second_pair_list.append((after_start_seconds, before_end_seconds))
    return month_second_pair_list
        
month_pair_list = get_each_month_in_a_list(after_start_date, before_end_date)
# print('month_pair_list: ', month_pair_list)
month_second_pair_list = get_each_month_second_in_a_list(month_pair_list)
# print('month_second_pair_list: ', month_second_pair_list)


# test mode
# after_start_date = date.fromisoformat('2016-04-01')
# before_end_date = date.fromisoformat('2016-04-30')
# after_start_seconds, before_end_seconds = get_start_end_seconds(after_start_date, before_end_date)
# print('after_start_seconds: ', after_start_seconds)
# print('before_end_seconds: ', before_end_seconds)

In [213]:
year_month_key_list = ['2016-04','2016-05','2016-06','2016-07','2016-08','2016-09','2016-10','2016-11','2016-12',
                       '2017-01','2017-02', '2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09',
                       '2017-10','2017-11','2017-12', '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', 
                       '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', 
                       '2019-02', '2019-03', '2019-04' ]
submission_year_month_dict = {}
for index in range(len(month_second_pair_list)): # initialization
    year_month = year_month_key_list[index]
    submission_year_month_dict[year_month] = {}
# for year_month in year_month_key_list: # initialization
#     submission_year_month_dict[year_month] = {}

In [214]:
# Category keyword
synonym_ecig = ['electronic-cigarette', 'electronic cigarette', 'electronic cig', ' e-cig', ' ecig', ' e cig', 'e-cigarette', 'ecigarette', ' e cigarette', 'e-juice', ' ejuice', ' e juice', 'e-liquid', ' eliquid', ' e liquid', ' e-smoke', ' esmoke', ' e smoke', ' vape', 'vaper', 'vaping', 'vape-juice',
                'vape-liquid', ' vapor', 'vaporizer', 'boxmod', 'cloud chaser', 'cloudchaser', 'smoke assist', 'ehookah', 'e-hookah', ' e hookah', 'stillblowingsmoke', 'still blowing smoke', 'cherry tip cigarillo', '#e-cig', '#ecig', '#e cigar', '#ejuice', '#e juice', '#eliquid', '#e liquid', '#vapor', '#ehookah', '#e hookah', 'smoke pod']
# remove ' fin '
brand_ecig = ['juul', 'vaporfi', 'vype pebble', 'v2 cig', 'v2cigs', 'v2 cigs', 'halocigs', ' njoy', 'markten', 'vuse', ' tryst', 'atomizer', 'cartomizer', 'south beach smoke', 'eversmoke', 'joye510', 'joye 510', 'joyetech', 'logicecig', 'smartsmoker', ' mistic', 'smokestiks', '21st century smoke', 'logic black label',
              ' finiti', 'nicotek', 'cigirex', 'ciga&blu', 'cig&blu', 'logic&cig', 'e-swisher', 'e swisher', 'eswisher', 'ezsmoker', 'ez&cig', 'green smoke', 'cigalectric', 'xhale o2', 'xhaleo2', 'cig2o', 'green smart living', 'greensmartliving', 'swisher blk', 'grimmgreen', '#njoy', '#fin ', '#finiti']
policy_ecig = ['sb140', 'sb 140', 'sb24', 'sb 24']
cessation_ecig = ['notblowingsmoke', 'not blowing smoke',
                  'tobaccofreekids', 'notareplacement']

def remove_space_and_conjunction(ecig_keywords):
    temp_keywords_list = []
    for keyword in ecig_keywords:
        keyword = keyword.strip()
        keyword = keyword.replace('&', ' ')
        keyword = keyword.replace('-', ' ')
        keyword = keyword.replace('#', '')
#         if ('&' in keyword) or ('-' in keyword) or ('#' in keyword):
#             continue
        temp_keywords_list.append(keyword)
    temp_keywords_list = list(set(temp_keywords_list)) # remove duplication
    return temp_keywords_list

# ecig keywords in total
ecig_keywords = []
ecig_keywords.extend(synonym_ecig)
ecig_keywords.extend(brand_ecig)
ecig_keywords.extend(policy_ecig)
ecig_keywords.extend(cessation_ecig)
ecig_keywords = remove_space_and_conjunction(ecig_keywords)


# San Francisco keywords. There is no difference between lower-case and upper-case
SF_keywords = ['san francisco', 'sf', 'sfo', 'san fran']

# San Francisco ecig keywords
SF_ecig_keywords = [i+' '+j for i in SF_keywords for j in ecig_keywords]
print('length of SF_ecig_keywords: ', len(SF_ecig_keywords))

length of SF_ecig_keywords:  328


In [215]:
def getPushshiftData_noSubreddit(query, after, before):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
    print(r)
    data = json.loads(r.text)
    return data['data']

In [216]:
def collect_submission(subm, year_month):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
#     author_fullName = subm['author_fullname']
    sub_id = subm['id']
    score = subm['score']
    try:
        subreddit = subm['subreddit']
    except KeyError:
        subreddit = "None"
        
    try:
        self_text = subm['selftext']
    except KeyError:
        self_text = "None"
        
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id, title, self_text, url, author, score,subreddit, created,numComms,permalink,flair))
#     subStats[sub_id] = subData
    submission_year_month_dict[year_month][sub_id] = subData
    

### Search keyword without specifying subreddit

In [217]:
month_second_pair_list

[(1459468800.0, 1461974400.0),
 (1462060800.0, 1464652800.0),
 (1464739200.0, 1467244800.0),
 (1467331200.0, 1469923200.0),
 (1470009600.0, 1472601600.0),
 (1472688000.0, 1475193600.0),
 (1475280000.0, 1477872000.0),
 (1477958400.0, 1480464000.0),
 (1480550400.0, 1483142400.0),
 (1483228800.0, 1485820800.0),
 (1485907200.0, 1488240000.0),
 (1488326400.0, 1490918400.0),
 (1491004800.0, 1493510400.0),
 (1493596800.0, 1496188800.0),
 (1496275200.0, 1498780800.0),
 (1498867200.0, 1501459200.0),
 (1501545600.0, 1504137600.0),
 (1504224000.0, 1506729600.0),
 (1506816000.0, 1509408000.0),
 (1509494400.0, 1512000000.0),
 (1512086400.0, 1514678400.0),
 (1514764800.0, 1517356800.0),
 (1517443200.0, 1519776000.0),
 (1519862400.0, 1522454400.0),
 (1522540800.0, 1525046400.0),
 (1525132800.0, 1527724800.0),
 (1527811200.0, 1530316800.0),
 (1530403200.0, 1532995200.0),
 (1533081600.0, 1535673600.0),
 (1535760000.0, 1538265600.0),
 (1538352000.0, 1540944000.0),
 (1541030400.0, 1543536000.0),
 (154362

In [219]:
subStats = {}

# go through each month
for index in range(len(month_second_pair_list)):
    after_start_month_second, before_end_month_second = month_second_pair_list[index]
    year_month = year_month_key_list[index]
    
    # go through search query
    for query in SF_ecig_keywords:
        print('--------------------------------------')
        print('Query: ', query)
        after = str(after_start_month_second).split('.')[0]
        before = str(before_end_month_second).split('.')[0]



        data = getPushshiftData_noSubreddit(query, after, before)
        # Will run until all posts have been gathered from the 'after' date up until before date
        while len(data) > 0:
            for submission in data:
    #             print(submission)
    #             print(submission['title'])
    #             print(submission['subreddit'])
    #             print('-----------')

                collect_submission(submission, year_month)

            # Prepare for next iteration
            print(len(data))
            print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
            after = data[-1]['created_utc']
            data = getPushshiftData_noSubreddit(query, after, before)

        print(len(data))
#         print('length of submission_year_month_dict.keys(): ', len(submission_year_month_dict.keys()))
        time.sleep(.400)


--------------------------------------
Query:  san francisco joye510
<Response [200]>
0
--------------------------------------
Query:  san francisco sb 140
<Response [200]>
0
--------------------------------------
Query:  san francisco green smoke
<Response [200]>
0
--------------------------------------
Query:  san francisco esmoke
<Response [200]>
0
--------------------------------------
Query:  san francisco v2cigs
<Response [200]>
0
--------------------------------------
Query:  san francisco xhaleo2
<Response [200]>
0
--------------------------------------
Query:  san francisco cloud chaser
<Response [200]>
0
--------------------------------------
Query:  san francisco logic black label
<Response [200]>
0
--------------------------------------
Query:  san francisco mistic
<Response [200]>
0
--------------------------------------
Query:  san francisco cigirex
<Response [200]>
0
--------------------------------------
Query:  san francisco halocigs
<Response [200]>
0
----------------

<Response [200]>
0
--------------------------------------
Query:  sf halocigs
<Response [200]>
0
--------------------------------------
Query:  sf logicecig
<Response [200]>
0
--------------------------------------
Query:  sf e cig
<Response [200]>
0
--------------------------------------
Query:  sf atomizer
<Response [200]>
0
--------------------------------------
Query:  sf sb24
<Response [200]>
0
--------------------------------------
Query:  sf smokestiks
<Response [200]>
0
--------------------------------------
Query:  sf markten
<Response [200]>
0
--------------------------------------
Query:  sf ehookah
<Response [200]>
0
--------------------------------------
Query:  sf south beach smoke
<Response [200]>
0
--------------------------------------
Query:  sf ecigarette
<Response [200]>
0
--------------------------------------
Query:  sf smartsmoker
<Response [200]>
0
--------------------------------------
Query:  sf ez cig
<Response [200]>
0
--------------------------------------


--------------------------------------
Query:  sfo electronic cig
<Response [200]>
0
--------------------------------------
Query:  sfo stillblowingsmoke
<Response [200]>
0
--------------------------------------
Query:  sfo cig2o
<Response [200]>
0
--------------------------------------
Query:  sfo tryst
<Response [200]>
0
--------------------------------------
Query:  sfo ejuice
<Response [200]>
0
--------------------------------------
Query:  sfo smoke pod
<Response [200]>
0
--------------------------------------
Query:  sfo sb 24
<Response [200]>
0
--------------------------------------
Query:  sfo e hookah
<Response [200]>
0
--------------------------------------
Query:  sfo vaping
<Response [200]>
0
--------------------------------------
Query:  sfo fin
<Response [200]>
0
--------------------------------------
Query:  sfo notblowingsmoke
<Response [200]>
0
--------------------------------------
Query:  sfo vape juice
<Response [200]>
0
--------------------------------------
Query:

--------------------------------------
Query:  san fran swisher blk
<Response [200]>
0
--------------------------------------
Query:  san fran tobaccofreekids
<Response [200]>
0
--------------------------------------
Query:  san fran smoke assist
<Response [200]>
0
--------------------------------------
Query:  san fran greensmartliving
<Response [200]>
0
--------------------------------------
Query:  san fran cloudchaser
<Response [200]>
0
--------------------------------------
Query:  san fran cherry tip cigarillo
<Response [200]>
0
--------------------------------------
Query:  san fran e cigar
<Response [200]>
0
--------------------------------------
Query:  san fran cartomizer
<Response [200]>
0
--------------------------------------
Query:  san fran xhale o2
<Response [200]>
0
--------------------------------------
Query:  san fran e juice
<Response [200]>
0
--------------------------------------
Query:  san fran e smoke
<Response [200]>
0
--------------------------------------
Q

--------------------------------------
Query:  san francisco e smoke
<Response [200]>
0
--------------------------------------
Query:  san francisco sb140
<Response [200]>
0
--------------------------------------
Query:  san francisco vuse
<Response [200]>
0
--------------------------------------
Query:  san francisco vaporizer
<Response [200]>
0
--------------------------------------
Query:  san francisco njoy
<Response [200]>
0
--------------------------------------
Query:  san francisco notareplacement
<Response [200]>
0
--------------------------------------
Query:  san francisco cigalectric
<Response [200]>
0
--------------------------------------
Query:  san francisco grimmgreen
<Response [200]>
0
--------------------------------------
Query:  san francisco eliquid
<Response [200]>
0
--------------------------------------
Query:  san francisco v2 cig
<Response [200]>
0
--------------------------------------
Query:  san francisco eversmoke
<Response [200]>
0
----------------------

...

--------------------------------------
Query:  sfo ez cig
<Response [200]>
0
--------------------------------------
Query:  sfo ciga blu
<Response [200]>
0
--------------------------------------
Query:  sfo finiti
<Response [200]>
0
--------------------------------------
Query:  sfo cig blu
<Response [200]>
0
--------------------------------------
Query:  sfo vaporfi
<Response [200]>
0
--------------------------------------
Query:  sfo ecig
<Response [200]>
0
--------------------------------------
Query:  sfo vaper
<Response [200]>
0
--------------------------------------
Query:  sfo nicotek
<Response [200]>
0
--------------------------------------
Query:  sfo joye 510
<Response [200]>
0
--------------------------------------
Query:  sfo electronic cigarette
<Response [200]>
0
--------------------------------------
Query:  sfo electronic cig
<Response [200]>
0
--------------------------------------
Query:  sfo stillblowingsmoke
<Response [200]>
0
--------------------------------------


--------------------------------------
Query:  san fran fin
<Response [200]>
0
--------------------------------------
Query:  san fran notblowingsmoke
<Response [200]>
0
--------------------------------------
Query:  san fran vape juice
<Response [200]>
0
--------------------------------------
Query:  san fran e liquid
<Response [200]>
0
--------------------------------------
Query:  san fran vape
<Response [200]>
0
--------------------------------------
Query:  san fran eswisher
<Response [200]>
0
--------------------------------------
Query:  san fran 21st century smoke
<Response [200]>
0
--------------------------------------
Query:  san fran still blowing smoke
<Response [200]>
0
--------------------------------------
Query:  san fran e swisher
<Response [200]>
0
--------------------------------------
Query:  san fran swisher blk
<Response [200]>
0
--------------------------------------
Query:  san fran tobaccofreekids
<Response [200]>
0
--------------------------------------
Query

--------------------------------------
Query:  san francisco smoke assist
<Response [200]>
0
--------------------------------------
Query:  san francisco greensmartliving
<Response [200]>
0
--------------------------------------
Query:  san francisco cloudchaser
<Response [200]>
0
--------------------------------------
Query:  san francisco cherry tip cigarillo
<Response [200]>
0
--------------------------------------
Query:  san francisco e cigar
<Response [200]>
0
--------------------------------------
Query:  san francisco cartomizer
<Response [200]>
0
--------------------------------------
Query:  san francisco xhale o2
<Response [200]>
0
--------------------------------------
Query:  san francisco e juice
<Response [200]>
0
--------------------------------------
Query:  san francisco e smoke
<Response [200]>
0
--------------------------------------
Query:  san francisco sb140
<Response [200]>
0
--------------------------------------
Query:  san francisco vuse
<Response [200]>
0
--

<Response [200]>
0
--------------------------------------
Query:  sf eliquid
<Response [200]>
0
--------------------------------------
Query:  sf v2 cig
<Response [200]>
0
--------------------------------------
Query:  sf eversmoke
<Response [200]>
0
--------------------------------------
Query:  sf juul
<Response [200]>
1
2019-04-25 21:06:26
<Response [200]>
0
--------------------------------------
Query:  sf v2 cigs
<Response [200]>
0
--------------------------------------
Query:  sf green smart living
<Response [200]>
0
--------------------------------------
Query:  sf joyetech
<Response [200]>
0
--------------------------------------
Query:  sf ezsmoker
<Response [200]>
0
--------------------------------------
Query:  sf vape liquid
<Response [200]>
0
--------------------------------------
Query:  sf vapor
<Response [200]>
0
--------------------------------------
Query:  sf vype pebble
<Response [200]>
0
--------------------------------------
Query:  sf e cigarette
<Response [200]>

--------------------------------------
Query:  san fran v2cigs
<Response [200]>
0
--------------------------------------
Query:  san fran xhaleo2
<Response [200]>
0
--------------------------------------
Query:  san fran cloud chaser
<Response [200]>
0
--------------------------------------
Query:  san fran logic black label
<Response [200]>
0
--------------------------------------
Query:  san fran mistic
<Response [200]>
0
--------------------------------------
Query:  san fran cigirex
<Response [200]>
0
--------------------------------------
Query:  san fran halocigs
<Response [200]>
0
--------------------------------------
Query:  san fran logicecig
<Response [200]>
0
--------------------------------------
Query:  san fran e cig
<Response [200]>
0
--------------------------------------
Query:  san fran atomizer
<Response [200]>
0
--------------------------------------
Query:  san fran sb24
<Response [200]>
0
--------------------------------------
Query:  san fran smokestiks
<Respons

In [220]:
submission_year_month_dict

{'2016-04': {'4fshu6': [('4fshu6',
    'Got robbed at gunpoint today in San Francisco (xpost /r/vaping)',
    "Gotta get this off my chest, and I feel like you guys will understand the extent of how much this sucks. My friend and I thought we'd take a trip to SF for 4/20 today to see the sights. We were walking back to our car through golden gate park at about 10pm when 3 guys came out of some bushes, put a gun to my chest and emptied my pockets. The only thing in my pocket was my brand new phone, but whatever, it's just stuff, right? The shitty thing is my vape was in my hand, which was over my head with my other hand because that's what they do on TV and I was about to soil my drawers, and of course they took that. They also took my friend's purse.\nIt could have been a lot worse and I'm so happy to be alive after that, honestly the biggest thing I'm feeling is relief. But the second biggest thing I'm feeling is a burning in my bones to relax with my sweet sig 150, who's been there f

In [221]:
for year_month in submission_year_month_dict:
    print(year_month + ":", len(submission_year_month_dict[year_month]))
    
# print('length of subStats.keys(): ', len(subStats.keys()))
# subStats.keys()

2016-04: 2
2016-05: 7
2016-06: 3
2016-07: 0
2016-08: 0
2016-09: 1
2016-10: 0
2016-11: 2
2016-12: 1
2017-01: 1
2017-02: 5
2017-03: 3
2017-04: 3
2017-05: 1
2017-06: 19
2017-07: 8
2017-08: 1
2017-09: 0
2017-10: 3
2017-11: 3
2017-12: 2
2018-01: 8
2018-02: 0
2018-03: 7
2018-04: 3
2018-05: 29
2018-06: 51
2018-07: 2
2018-08: 6
2018-09: 4
2018-10: 3
2018-11: 7
2018-12: 3
2019-01: 2
2019-02: 2
2019-03: 28
2019-04: 12


In [222]:
def save_submission_year_month_dict_to_file(fileName, submission_year_month_dict):
    location = "/mnt/volume-second-5T/subreddits/"
    
    for year_month in submission_year_month_dict.keys():
        length_count = 0
        temp_fileName = fileName + "_" + year_month + ".csv"
        print(temp_fileName)
        file = location + temp_fileName
        with open(file, 'w', newline='', encoding='utf-8') as file: 
            a = csv.writer(file, delimiter=',')
            headers = ["Post_ID","Title","Self_text", "Url","Author","Score","Subreddit","Publish_date","Number_of_comments","Permalink","Flair"]
            a.writerow(headers)
            for sub in submission_year_month_dict[year_month]:
                a.writerow(submission_year_month_dict[year_month][sub][0])
                length_count+=1

            print(str(length_count) + " submissions have been saved")
        
save_submission_year_month_dict_to_file("place_and_ecig_keywords_collection", submission_year_month_dict)

place_and_ecig_keywords_collection_2016-04.csv
2 submissions have been saved
place_and_ecig_keywords_collection_2016-05.csv
7 submissions have been saved
place_and_ecig_keywords_collection_2016-06.csv
3 submissions have been saved
place_and_ecig_keywords_collection_2016-07.csv
0 submissions have been saved
place_and_ecig_keywords_collection_2016-08.csv
0 submissions have been saved
place_and_ecig_keywords_collection_2016-09.csv
1 submissions have been saved
place_and_ecig_keywords_collection_2016-10.csv
0 submissions have been saved
place_and_ecig_keywords_collection_2016-11.csv
2 submissions have been saved
place_and_ecig_keywords_collection_2016-12.csv
1 submissions have been saved
place_and_ecig_keywords_collection_2017-01.csv
1 submissions have been saved
place_and_ecig_keywords_collection_2017-02.csv
5 submissions have been saved
place_and_ecig_keywords_collection_2017-03.csv
3 submissions have been saved
place_and_ecig_keywords_collection_2017-04.csv
3 submissions have been saved